In [42]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras


In [43]:
df=pd.read_csv("https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/examples/AMAZON-REVIEW-DATA-CLASSIFICATION.csv")

In [44]:
df.head()

,reviewText,summary,verified,time,log_votes,isPositive
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",IDEAL FOR BEGINNER!,True,1361836800,0.000000,1.0
1,unable to open or use,Two Stars,True,1452643200,0.000000,0.0
2,Waste of money!!! It wouldn't load to my system.,Dont buy it!,True,1433289600,0.000000,0.0
3,I attempted to install this OS on two differen...,I attempted to install this OS on two differen...,True,1518912000,0.000000,0.0
4,I've spent 14 fruitless hours over the past tw...,Do NOT Download.,True,1441929600,1.098612,0.0


In [45]:
df=df[["reviewText","isPositive"]]

In [46]:
df.head()

,reviewText,isPositive
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",1.0
1,unable to open or use,0.0
2,Waste of money!!! It wouldn't load to my system.,0.0
3,I attempted to install this OS on two differen...,0.0
4,I've spent 14 fruitless hours over the past tw...,0.0


In [47]:
df["isPositive"]=df["isPositive"].astype(int)

<ipython-input-47-8801bec47716>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["isPositive"]=df["isPositive"].astype(int)


In [48]:
df.head()

,reviewText,isPositive
0,"PURCHASED FOR YOUNGSTER WHO\nINHERITED MY ""TOO...",1
1,unable to open or use,0
2,Waste of money!!! It wouldn't load to my system.,0
3,I attempted to install this OS on two differen...,0
4,I've spent 14 fruitless hours over the past tw...,0


In [49]:
df["reviewText"][4]

"I've spent 14 fruitless hours over the past two days fruitlessly attempting to install this software on my computer and nothing I've found has worked. I need the software to type proficiently due to disability, and it will not install. The download itself seems to be a corrupted file, I have a fair amount of computer skills and no amount of tinkering has made the program work, so, judging by other reviews, it must be the program itself."

In [50]:
df=df[~df["reviewText"].isnull()]

In [51]:
df.isnull().sum()

reviewText    0
isPositive    0
dtype: int64

In [52]:
df["reviewText"]=df["reviewText"].apply(lambda x: one_hot(x,5000))

In [53]:
df.head()

,reviewText,isPositive
0,"[1491, 3502, 4964, 4836, 1861, 3565, 23, 4401,...",1
1,"[1483, 2959, 1816, 346, 1289]",0
2,"[4599, 3205, 4343, 2200, 3262, 2021, 2959, 356...",0
3,"[4217, 48, 2959, 3262, 378, 2984, 718, 1161, 4...",0
4,"[1770, 2630, 3954, 3396, 2650, 986, 3419, 3684...",0


In [54]:
padded_reviws=pad_sequences(df["reviewText"],maxlen=100)

In [55]:
padded_reviws

array([[   0,    0,    0, ...,  413,  378, 2562],
       [   0,    0,    0, ..., 1816,  346, 1289],
       [   0,    0,    0, ..., 2959, 3565, 4534],
       ...,
       [   0,    0,    0, ..., 4246, 2451, 2631],
       [   0,    0,    0, ...,  594, 4240,  775],
       [   0,    0,    0, ..., 2660, 1960,   64]], dtype=int32)

In [56]:
model=keras.models.Sequential([
   keras.layers.Embedding(5000,16,input_length=100),
   keras.layers.Flatten(),
   keras.layers.Dense(1,activation="sigmoid")


])

In [57]:
model.compile(optimizer="Adam",loss=keras.losses.binary_crossentropy,metrics=["accuracy"])

In [58]:
model.fit(padded_reviws,df["isPositive"],epochs=10)

Epoch 1/10
2188/2188 [==============================] - 40s 18ms/step - loss: 0.4095 - accuracy: 0.8126
Epoch 2/10
2188/2188 [==============================] - 11s 5ms/step - loss: 0.3302 - accuracy: 0.8592
Epoch 3/10
2188/2188 [==============================] - 10s 5ms/step - loss: 0.2999 - accuracy: 0.8757
Epoch 4/10
2188/2188 [==============================] - 9s 4ms/step - loss: 0.2654 - accuracy: 0.8929
Epoch 5/10
2188/2188 [==============================] - 9s 4ms/step - loss: 0.2271 - accuracy: 0.9138
Epoch 6/10
2188/2188 [==============================] - 9s 4ms/step - loss: 0.1904 - accuracy: 0.9311
Epoch 7/10
2188/2188 [==============================] - 8s 4ms/step - loss: 0.1585 - accuracy: 0.9467
Epoch 8/10
2188/2188 [==============================] - 8s 4ms/step - loss: 0.1331 - accuracy: 0.9568
Epoch 9/10
2188/2188 [==============================] - 9s 4ms/step - loss: 0.1120 - accuracy: 0.9651
Epoch 10/10
2188/2188 [==============================] - 8s 4ms/step - loss: 0

In [59]:
model.get_weights() #Word embeddings

[array([[-0.00062943,  0.00163935, -0.00636782, ..., -0.00822713,
         -0.00613947, -0.00269842],
        [-0.06004436, -0.21112186,  0.3663721 , ...,  0.56364787,
         -0.33856782, -0.44279647],
        [-0.38919672,  0.48067603, -0.3326358 , ..., -0.26887926,
         -0.0525478 ,  0.2959233 ],
        ...,
        [ 0.28378868, -0.22849585,  0.4873241 , ...,  0.43429914,
         -0.13619755, -0.48415095],
        [-0.18408214,  0.08112732,  0.15242858, ...,  0.23374704,
          0.1967982 ,  0.32702592],
        [ 0.02403962,  0.31421122, -0.08477927, ..., -0.10008597,
          0.0651952 , -0.19730757]], dtype=float32),
 array([[-0.36614588],
        [ 0.56404936],
        [-0.33505413],
        ...,
        [ 1.202424  ],
        [-0.96264017],
        [ 0.1477543 ]], dtype=float32),
 array([-0.02788941], dtype=float32)]

In [63]:
model.predict(padded_reviws)

2188/2188 [==============================] - 4s 2ms/step


array([[0.980996  ],
       [0.05321351],
       [0.0044031 ],
       ...,
       [0.9953353 ],
       [0.7856448 ],
       [0.9967464 ]], dtype=float32)